# Testeando el padData y el createIndexation

## Paquetes necesarios

In [1]:
import transformData3 as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t

## Definición del conjunto de datos

### Datos sin transformación

In [6]:
SHAPE = [124, 129, 346]

SIZE = np.prod(SHAPE)

src = np.arange(SIZE, dtype=np.int32).reshape(SHAPE)

### Datos transformados

In [7]:
PART_SHAPE = [32, 32, 32]

src_part, indexation = td.tData(src, PART_SHAPE, inverse=False)